## 必要なライブラリのimport

In [4]:
import numpy as np
import pylab as plt
import matplotlib as mpl
import glob
import csv
import os
import netCDF4 as nc
import conv_NC4files as cnc
import pandas as pd

## .ncファイルの読み込みと変換

In [7]:
infile = '.nc保存先のPATH（任意）/.ncファイル名'

#ncファイルの読み込み
nc0  = nc.Dataset(infile, 'r', encoding='SHIFT_JIS')
nc0v = nc0['PRODUCT'].variables

#ncファイル -> csvファイルへ-----------------------------------------------

indir  = '.nc保存先のPATH（任意）'
outdir = '.csvに変換後の出力先のPATH（任意）'
outdir1 = outdir + 'SO2/nc4csv00/'

if not os.path.exists(outdir1):
    os.makedirs(outdir1)

flist  = glob.glob(indir + 'S5P_OFFL_L2__SO2____*')
flist.sort()

In [8]:
#Run conversion
optkeys = ['sulfurdioxide_total_vertical_column', 'sulfurdioxide_total_vertical_column_precision']
for infile in flist:
    fname   = infile.split('\\')[-1].split('.nc')[0]
    outfile = outdir1+'nc4csv00_'+fname+'.csv'
    cnc.conv_s5p_csv(infile, outfile, optkeys)

## 空行の削除と値のフィルタリング

In [9]:
for file in glob.glob(outdir1 +'nc4csv00_S5P_OFFL_L2__SO2____*'):
  　 df = pd.read_csv(file)
   
    #偶数行は空なので奇数行のみ抽出
    df = df[1::2]
    
    #qa_value 0.75より大きい値を抽出。精度が高いため。
    df = df[df['qa_value'] > 0.5]
   
   
    file = file.split('nc4csv00_S5P_OFFL_L2__SO2____')[-1]
   
    outfile ='フィルタリング後任意の出力先PATH' + 'SO2_out_scr_' + file 
  
    df.to_csv(outfile)

## Plotに必要なライブラリのimport

In [ ]:
import matplotlib.pyplot as plt
%matplotlib 
import numpy as np
import glob
import os 
os.environ["PROJ_LIB"] = "C:/Users/ユーザー名/anaconda3/pkgs/proj-7.1.0-h7d85306_1/Library/share"; 
from mpl_toolkits.basemap import Basemap, cm
fig = plt.figure(figsize=(10,10))
#plt.title('yyyy/mm/dd')

#Basemap instanceの作成、緯度経度を西之島に合わせる
#緯度経度の範囲は自分の好きなようにカスタマイズする
m = Basemap(projection='merc',
resolution='h',
llcrnrlon=135,
llcrnrlat=25,
urcrnrlon=155,
urcrnrlat=37)


#海岸線を描く
m.drawcoastlines()


## 緯度、経度、SO2カラム量の抽出

In [ ]:
Dir = 'フィルタリング後の.csv出力のPATH'        
files = glob.glob(Dir + '/*.csv'); files.sort()
lon_total = []
lat_total = []
SO2_total = []
#SO2の単位はDU(Dobson Unit)
#DU = 2241.15
mol = 6.00214*1.e+4
for file in files:
    d = np.loadtxt(file,delimiter=',',skiprows=1, unpack=True)
    lon = d[2,:]
    lat = d[3,:]
    SO2 = mol*d[5,:]
    lon_total.extend(lon)
    lat_total.extend(lat)
    SO2_total.extend(SO2)

## SO2濃度の調整など

In [ ]:
lon, lat = m(lon_total, lat_total)

#火山や山火事などSO2が大量に放出する場合vmax=20にするとSO2濃度が鮮明になる
im=m.scatter(lon,lat,alpha=0.9,s=1,vmin=0.,vmax=2.e+1,
c=SO2_total, edgecolors='none', linewidths=1.5,cmap="jet")

cbar = m.colorbar(im,location='bottom',pad="5%")
cbar.set_label('$10^{15}molec/cm^2$')

plt.show()